In [2]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score



ModuleNotFoundError: No module named 'pandas'

In [18]:
from flaml import AutoML

In [36]:
data = pd.read_csv('train.csv')

In [95]:
y = data["Survived"]
X = data.drop("Survived",axis=1)


In [96]:
X

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [55]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [97]:
if len(y.unique()) > 10: # if more than 10 unique target values, treat as regression
    problem_type = 'regression'
    automl_settings = {
    "time_budget": 60,  # total running time in seconds
    "metric": "r2",  # primary metrics can be chosen from: ['accuracy', 'roc_auc', 'roc_auc_weighted', 'roc_auc_ovr', 'roc_auc_ovo', 'f1', 'log_loss', 'mae', 'mse', 'r2'] Check the documentation for more details (https://microsoft.github.io/FLAML/docs/Use-Cases/Task-Oriented-AutoML#optimization-metric)
    "task": "regerssion",  # task type
    "estimator_list": ["xgboost", "catboost", "lgbm"],
    "log_file_name": "flaml.log",  # flaml log file
}
else:
    automl_settings = {
    "time_budget": 60,  # total running time in seconds
    "metric": "accuracy",  # primary metrics can be chosen from: ['accuracy', 'roc_auc', 'roc_auc_weighted', 'roc_auc_ovr', 'roc_auc_ovo', 'f1', 'log_loss', 'mae', 'mse', 'r2'] Check the documentation for more details (https://microsoft.github.io/FLAML/docs/Use-Cases/Task-Oriented-AutoML#optimization-metric)
    "task": "classification",  # task type
    "estimator_list": ["xgboost", "catboost", "lgbm"],
    "log_file_name": "flaml.log",  # flaml log file
}
    problem_type = 'classification'

In [98]:
numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessing = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, X.select_dtypes(include=['int64', 'float64']).columns),
        ('cat', categorical_transformer, X.select_dtypes(include=['object']).columns)
    ])

In [99]:
# Start the search for the best pipeline
automl = AutoML()


In [100]:
automl_pipeline = Pipeline([
    ("Preprocessing",preprocessing),
    ("automl", automl)
])

In [101]:
search_space = {
    'n_estimators': [100, 200, 300],
    'max_depth': [2, 4, 6],
    'learning_rate': [0.1, 0.2, 0.3],
    'feature_fraction': [0.5, 0.7, 1.0]
}

In [102]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [109]:

pipeline_settings = {
    f"automl__{key}": value for key, value in automl_settings.items()
}
k = automl_pipeline.fit(X_train, y_train, **pipeline_settings)

[flaml.automl.logger: 04-30 06:32:28] {1682} INFO - task = classification
[flaml.automl.logger: 04-30 06:32:28] {1689} INFO - Data split method: stratified
[flaml.automl.logger: 04-30 06:32:28] {1692} INFO - Evaluation method: holdout
[flaml.automl.logger: 04-30 06:32:28] {1790} INFO - Minimizing error metric: 1-accuracy
[flaml.automl.logger: 04-30 06:32:28] {1900} INFO - List of ML learners in AutoML Run: ['xgboost', 'catboost', 'lgbm']
[flaml.automl.logger: 04-30 06:32:28] {2210} INFO - iteration 0, current learner xgboost
[flaml.automl.logger: 04-30 06:32:28] {2336} INFO - Estimated sufficient time budget=340s. Estimated necessary time budget=0s.
[flaml.automl.logger: 04-30 06:32:28] {2383} INFO -  at 0.1s,	estimator xgboost's best error=0.1644,	best estimator xgboost's best error=0.1644
[flaml.automl.logger: 04-30 06:32:28] {2210} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 04-30 06:32:28] {2383} INFO -  at 0.1s,	estimator lgbm's best error=0.1781,	best estimator

c:\Users\raji9\OneDrive\Desktop\New folder\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\raji9\OneDrive\Desktop\New folder\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\raji9\OneDrive\Desktop\New folder\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.logger: 04-30 06:32:28] {2210} INFO - iteration 5, current learner xgboost
[flaml.automl.logger: 04-30 06:32:28] {2383} INFO -  at 0.3s,	estimator xgboost's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:32:28] {2210} INFO - iteration 6, current learner xgboost
[flaml.automl.logger: 04-30 06:32:28] {2383} INFO -  at 0.3s,	estimator xgboost's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:32:28] {2210} INFO - iteration 7, current learner xgboost
[flaml.automl.logger: 04-30 06:32:28] {2383} INFO -  at 0.3s,	estimator xgboost's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:32:28] {2210} INFO - iteration 8, current learner xgboost
[flaml.automl.logger: 04-30 06:32:28] {2383} INFO -  at 0.4s,	estimator xgboost's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:32:28] {2210} INFO - iteration 9, cur

c:\Users\raji9\OneDrive\Desktop\New folder\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\raji9\OneDrive\Desktop\New folder\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\raji9\OneDrive\Desktop\New folder\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\raji9\OneDrive\Desktop\New folder\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.logger: 04-30 06:32:28] {2383} INFO -  at 0.4s,	estimator lgbm's best error=0.1781,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:32:28] {2210} INFO - iteration 10, current learner lgbm
[flaml.automl.logger: 04-30 06:32:28] {2383} INFO -  at 0.5s,	estimator lgbm's best error=0.1644,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:32:28] {2210} INFO - iteration 11, current learner xgboost
[flaml.automl.logger: 04-30 06:32:28] {2383} INFO -  at 0.6s,	estimator xgboost's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:32:28] {2210} INFO - iteration 12, current learner xgboost
[flaml.automl.logger: 04-30 06:32:28] {2383} INFO -  at 0.6s,	estimator xgboost's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:32:28] {2210} INFO - iteration 13, current learner lgbm


...

c:\Users\raji9\OneDrive\Desktop\New folder\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.logger: 04-30 06:33:27] {2383} INFO -  at 59.6s,	estimator catboost's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:33:27] {2210} INFO - iteration 337, current learner lgbm
[flaml.automl.logger: 04-30 06:33:27] {2383} INFO -  at 59.6s,	estimator lgbm's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:33:27] {2210} INFO - iteration 338, current learner xgboost
[flaml.automl.logger: 04-30 06:33:27] {2383} INFO -  at 59.7s,	estimator xgboost's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:33:27] {2210} INFO - iteration 339, current learner xgboost
[flaml.automl.logger: 04-30 06:33:27] {2383} INFO -  at 59.7s,	estimator xgboost's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:33:27] {2210} INFO - iteration 340, current learner lgbm
[flaml.automl.logger: 04-30 06:33:28] {2383} INFO -  at 59.7s,	e

c:\Users\raji9\OneDrive\Desktop\New folder\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\raji9\OneDrive\Desktop\New folder\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.logger: 04-30 06:33:28] {2383} INFO -  at 59.9s,	estimator lgbm's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:33:28] {2210} INFO - iteration 342, current learner lgbm
[flaml.automl.logger: 04-30 06:33:28] {2383} INFO -  at 60.0s,	estimator lgbm's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:33:28] {2210} INFO - iteration 343, current learner xgboost
[flaml.automl.logger: 04-30 06:33:28] {2383} INFO -  at 60.0s,	estimator xgboost's best error=0.1507,	best estimator xgboost's best error=0.1507
[flaml.automl.logger: 04-30 06:33:28] {2619} INFO - retrain xgboost for 0.0s
[flaml.automl.logger: 04-30 06:33:28] {2622} INFO - retrained model: XGBClassifier(base_score=None, booster=None, callbacks=[],
              colsample_bylevel=0.8148474110627004, colsample_bynode=None,
              colsample_bytree=0.9777234800442423, early_stopping_rounds=None,
              enable_categorical=

c:\Users\raji9\OneDrive\Desktop\New folder\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\raji9\OneDrive\Desktop\New folder\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [132]:
automl_pipeline.steps[1][1].best_estimator


'xgboost'

In [128]:
x = automl_pipeline.predict(X_test)

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1])

In [112]:
automl = automl_pipeline.steps[1][1]
# Get the best config and best learner
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1 - automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 4, 'max_leaves': 4, 'min_child_weight': 3.815612027960909, 'learning_rate': 0.03859136192132085, 'subsample': 1.0, 'colsample_bylevel': 0.8148474110627004, 'colsample_bytree': 0.9777234800442423, 'reg_alpha': 0.0009765625, 'reg_lambda': 5.525802807180917}
Best accuracy on validation data: 0.8493
Training duration of best run: 0.027 s


In [144]:
y_pred1 = automl_pipeline.predict(X_test)

In [155]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [146]:
confusion_matrix(y_pred1,y_test)

array([[88, 22],
       [17, 52]], dtype=int64)

In [154]:
accuracy_score(y_pred1,y_test)

0.7821229050279329

In [157]:
print(classification_report(y_pred1,y_test))

              precision    recall  f1-score   support

           0       0.84      0.80      0.82       110
           1       0.70      0.75      0.73        69

    accuracy                           0.78       179
   macro avg       0.77      0.78      0.77       179
weighted avg       0.79      0.78      0.78       179



In [140]:
test = pd.read_csv("test.csv")

In [142]:
y_pred = automl_pipeline.predict(test)

'xgboost'